In [3]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv('/content/data.csv')

In [ ]:
!pip install sentence-transformers

In [29]:
sen = data.iloc[:,1]

In [30]:
sen

0     After two years of lockdowns and COVID-19 surg...
1     @ManozKumarTalks I hope this won't affect movi...
2     @globaltimesnews May be much more than 4 milli...
3     India objects to WHO's methodology to calculat...
4     I hope this won't affect movies now.\n\nDelhi ...
                            ...                        
92    Musk has long advocated a libertarian vision o...
93    Twitter’s <a href="https://www.reuters.com/com...
94    Analysis: Musk’s past musings about Twitter sh...
95    Twitter <a href="https://www.reuters.com/compa...
96    The Tesla boss now has a 9.2% stake in the soc...
Name: Tweet, Length: 97, dtype: object

In [31]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
sen_embeddings = model.encode(sen)
sen_embeddings.shape

(97, 768)

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
cosine_similarity(
    [sen_embeddings[1],sen_embeddings[4]],
    [sen_embeddings[0],sen_embeddings[2]]
)

array([[0.6185789 , 0.66299814],
       [0.6045799 , 0.65342605]], dtype=float32)

In [40]:
cosine_similarity(
    [sen_embeddings[1]],
    [sen_embeddings[0],sen_embeddings[2]]
)

array([[0.6185789, 0.6629982]], dtype=float32)

In [33]:
similarity_df = pd.DataFrame(columns = ['from_article','to_article','similarity_score'])

In [34]:
from tqdm import tqdm


In [47]:
cos_sim = cosine_similarity(sen_embeddings,sen_embeddings)


In [54]:
len = data.shape[0]
for i in range(len):
  for j in range(len):
    similarity_df = similarity_df.append({'from_article':i,'to_article':j,'similarity_score':cos_sim[i][j]},ignore_index=True)

In [55]:
similarity_df

,from_article,to_article,similarity_score
0,0.0,0.0,1.000000
1,0.0,1.0,0.618579
2,0.0,2.0,0.468383
3,0.0,3.0,0.527573
4,0.0,4.0,0.604580
...,...,...,...
9404,96.0,92.0,0.542154
9405,96.0,93.0,0.641532
9406,96.0,94.0,0.795119
9407,96.0,95.0,0.744858


In [62]:
similarity_df['from_article'] = similarity_df['from_article'].astype('int32')
similarity_df['to_article'] = similarity_df['to_article'].astype('int32')

In [65]:
similarity_df[similarity_df['from_article'] == 1].head(10)

,from_article,to_article,similarity_score
97,1,0,0.618579
98,1,1,1.000000
99,1,2,0.662998
100,1,3,0.457670
101,1,4,0.994338
102,1,5,0.337801
103,1,6,0.569384
104,1,7,0.847917
105,1,8,0.856060
106,1,9,0.368901


In [76]:
similarity_df.to_csv('similarity_scores.csv',index = False)

In [69]:
data = data.reset_index()

In [73]:
data = data.rename({'index':'article_code'},axis = 1)

In [75]:
data.to_csv('data.csv',index = False)